In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_20.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

def time_delayed_cross_correlation(df, col1, col2, max_tau):
    """
    Computes the time-delayed cross-correlation function for two time series N_i and N_j.
    
    Args:
    N_i (numpy array): Time series for node i.
    N_j (numpy array): Time series for node j.
    max_tau (int): Maximum time delay (positive or negative).
    
    Returns:
    cross_corrs (numpy array): Cross-correlation values for each time delay in the range [-max_tau, max_tau].
    time_lags (numpy array): Corresponding time lags.
    """
    n = len(col1)
    mean_i = df[col1].mean()
    mean_j = df[col2].mean()
    
    # Prepare arrays for cross-correlations and time lags
    cross_corrs = []
    time_lags = range(-max_tau, max_tau + 1)
    
    for tau in time_lags:
        if tau < 0:
            shifted_j = np.roll(df[col2], tau)  # shift N_j forward (N_j(t+τ))
            valid_indices = np.arange(-tau, n)  # indices where the time shift is valid
        else:
            shifted_i = np.roll(df[col1], -tau)  # shift N_i backward (N_i(t-τ))
            valid_indices = np.arange(0, n - tau)  # indices where the time shift is valid

        # Calculate the cross-correlation for the valid time points
        if tau < 0:
            numerator = np.mean(df[col1][valid_indices] * shifted_j[valid_indices]) - mean_i * mean_j
            denominator = np.std(df[col1][valid_indices]) * np.std(shifted_j[valid_indices])
        else:
            numerator = np.mean(shifted_i[valid_indices] * df[col2][valid_indices]) - mean_i * mean_j
            denominator = np.std(shifted_i[valid_indices]) * np.std(df[col2][valid_indices])
        
        # Handle the case where the denominator is zero
        if denominator != 0:
            corr = numerator / denominator
        else:
            corr = 0
        
        cross_corrs.append(corr)    
    max_corr = np.max(cross_corrs)
    if max_corr >= threshold:
        return max_corr
    else:
        return 0

#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename):
    submatrix = pd.DataFrame(index=columns, columns=columns)
    for col1, col2 in product(columns, repeat=2):
        submatrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    submatrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix{filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 5105 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

'''
#Now we will try to get all these 16 submatricese (dividing the columns into 4 groups)
#Let's fix the group sizes
group_sizes = [250, 250, 250, 254]

#Divide columns into groups
groups = divide_into_groups(data, group_sizes)

#compute submatrices on the diagonal
for i, group in enumerate(groups, start=1):
    compute_submatrix(data, group, f'submatrix_group_{i}.csv')

#compute submatrices out of the diagonal
for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        compute_cross_group_matrix(data, groups[i], groups[j], f'cross_matrix_group_{i+1}_group_{j+1}.csv')
    
'''